# Capítulo 6 - Algoritmos com Modelo


Você pode rodar este notebook localmente ou no Colab. Para abrir diretamente no Colab, basta clicar no link abaixo.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap06/cap06-main-1.ipynb)

## Configurações Iniciais

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg freeglut3-dev xvfb

    !pip install gym==0.23.1
    !pip install optuna

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

clear_output()

In [ ]:
if IN_COLAB:
    # Set up fake display; otherwise rendering will fail
    import os
    os.system("Xvfb :1 -screen 0 1024x768x24 &")
    os.environ['DISPLAY'] = ':1'

    from util.notebook import display_videos_from_path

In [ ]:
import random as rand

import gym
import numpy as np
import optuna

from util.experiments import repeated_exec, test_greedy_Q_policy
from util.plot import plot_result, plot_multiple_results
from util.notebook import display_videos_from_path

from util.envs import RacetrackEnv

## 1 - Q-Learning

O *Q-Learning* é um algoritmo **livre de modelo**, mas foi reproduzido aqui para compararmos com:

*  O código do próximo algoritmo
*  O desempenho do próximo algoritmo

Ambos os algoritmos serão executados (em um teste simples) com o ambiente e os parâmetros dados abaixo:

In [ ]:
# escolha o ambiente descomentando uma das linhas abaixo
#ENV_NAME = 'taxi'
ENV_NAME = 'racetrack'

LR = 0.3
GAMMA = 0.90
EPSILON = 0.1

VERBOSE = True

Segue o mesmo código do *Q-learning** que vimos antes (ainda com o critério de parada baseado na quantidade de episódios).

In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def epsilon_greedy(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])

In [ ]:
def run_qlearning(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q com valores aleatórios de -1.0 a 0.0
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape=(env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            if done:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table
            # delta = (estimativa usando a nova recompensa) - estimativa antiga
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if VERBOSE and ((i+1) % 100 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_ep, Q

In [ ]:
if ENV_NAME == 'taxi':
    rmax = 10.0
    env = gym.make("Taxi-v3")
    EPISODES = 700
else:
    rmax = None
    env = RacetrackEnv()
    EPISODES = 2_000

rewards1, qtable1 = run_qlearning(env, EPISODES, LR, GAMMA, EPSILON)
print("Últimos resultados: media =", np.mean(rewards1[-20:]), ", desvio padrao =", np.std(rewards1[-20:]))

In [ ]:
test_greedy_Q_policy(env, qtable1, 2, render=False, recorded_video_folder='./videos-qlearn')
display_videos_from_path('./videos-qlearn', speed=0.5)

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards1, rmax, cumulative=False, window=30)

## 2 - Dyna-Q

O *Dyna-Q* é um algoritmo **com modelo** que é uma extensão do *Q-Learning*. Compare os códigos para entender o Dyna-Q.

In [ ]:
def planning(model, planning_steps, Q, lr, gamma):
    all_s_a = list(model.keys())
    if len(all_s_a) < planning_steps:
        samples = rand.choices(all_s_a, k=planning_steps)
    else:
        samples = rand.sample(all_s_a, k=planning_steps)

    for s, a in samples:
        r, next_s, is_terminal = model[(s,a)]
        if is_terminal:
            V_next_s = 0
        else:
            V_next_s = np.max(Q[next_s])
        delta = (r + gamma * V_next_s) - Q[s,a]
        Q[s,a] = Q[s,a] + lr * delta

In [ ]:
# Algoritmo Dyna Q
def run_dyna_q(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, planning_steps=5):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q
    Q = np.zeros(shape=(env.observation_space.n, num_actions))

    model = dict({})

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            if done:
                # para estados terminais
                V_next_state = 0
                next_state = env.reset()
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table / direct RL
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            # atualiza o modelo
            model[state,action] = (reward, next_state, done)

            # planejamento / indirect RL
            planning(model, planning_steps, Q, lr, gamma)

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 1000 passos, imprime informação sobre o progresso
        if VERBOSE and ((i+1) % 1000 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Step {i+1} Average Reward (last 100): {avg_reward:.3f}")

    state = env.reset()
    reward = 0

    return sum_rewards_per_ep, Q

In [ ]:
if ENV_NAME == 'taxi':
    rmax = 10.0
    env = gym.make("Taxi-v3")
    EPISODES = 700
else:
    rmax = None
    env = RacetrackEnv()
    EPISODES = 2_000

rewards2, qtable2 = run_dyna_q(env, EPISODES, LR, GAMMA, EPSILON, planning_steps=5)
print("Últimos resultados: media =", np.mean(rewards2[-20:]), ", desvio padrao =", np.std(rewards2[-20:]))

In [ ]:
# Faz alguns testes, usando a tabela de forma greedy e mostra o vídeo
test_greedy_Q_policy(env, qtable2, 2, render=False, recorded_video_folder='./videos-dynaq')
display_videos_from_path('./videos-dynaq', speed=0.5)

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards2, rmax, cumulative='no', window=30)

## 3 - Experimentos com "Taxi-v3" (ou o "Racetrack")

Nesta seção, você pode fazer experimentos no `Taxi-v3` ou no `Racetrack`, como preferir. Porém, sabia que o treinamento com Racetrack é bem mais demorado.

Escolha o ambiente na célula de código abaixo.

Os experimentos têm duas partes:
- Na seção 3.1, vamos otimizar os parâmetros do Q-Learning
- Na seção 3.2, vamos rodar o Q-Learning e o Dyna-Q (ambos com os parâmetros do Q-Learning), para comparar


In [ ]:
# Escolha abaixo o ambiente
ENV_NAME_EXPERIMENT = 'taxi'
#ENV_NAME_EXPERIMENT = 'racetrack'

### 3.1 - Otimizando Parâmetros do Q-Learning

In [ ]:
if ENV_NAME_EXPERIMENT == 'taxi':
    ENV = gym.make("Taxi-v3")
    EPISODES_PER_TRIAL = 200
else:
    ENV = RacetrackEnv()
    EPISODES_PER_TRIAL = 800

RUNS_PER_TRIAL = 3

# Para omitir as saídas intermediárias dos algoritmos
VERBOSE = False

In [ ]:
def train(trial : optuna.Trial):
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    lr = trial.suggest_uniform('lr', 0.1, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    gamma = trial.suggest_uniform('gamma', 0.5, 1.0)

    print(f"\nTRIAL #{trial.number}: lr={lr}, eps={eps}, gamma={gamma}")

    # roda o algoritmo várias vezes
    global ENV, RUNS_PER_TRIAL, EPISODES_PER_TRIAL
    results = repeated_exec(RUNS_PER_TRIAL, "qlearn-optuna", run_qlearning, ENV, EPISODES_PER_TRIAL, lr=lr, epsilon=eps, gamma=gamma)

    # soma dos retornos não-descontado finais (dos últimos 20 episódios)
    return np.sum(results[1][:,-20:])

In [ ]:
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_planning.db',
                        study_name=f'qlearning_{ENV_NAME_EXPERIMENT}',
                        load_if_exists=True)

study.optimize(train, n_trials=30)
clear_output()

print(f"MELHORES PARÂMETROS PARA {ENV_NAME_EXPERIMENT}:")
print(study.best_params)
qlearn_params_taxi = study.best_params

### 3.2 - Experimentos Comparativos

In [ ]:
if ENV_NAME_EXPERIMENT == 'taxi':
    environment = gym.make("Taxi-v3")
    EPISODES = 800
    RUNS  = 10
else:
    environment = RacetrackEnv()
    EPISODES = 4_000
    RUNS = 3

AUTO_LOAD = True

results_t = []

results_t.append( repeated_exec(RUNS, f"Q-Learning ", run_qlearning, environment, EPISODES, **qlearn_params_taxi, auto_load=AUTO_LOAD) )
clear_output()

#plan_steps = 1
#results_t.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passo)", run_dyna_q, environment, EPISODES, **qlearn_params_taxi, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
#clear_output()

plan_steps = 10
results_t.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passos)", run_dyna_q, environment, EPISODES, **qlearn_params_taxi, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
clear_output()


In [ ]:
plot_multiple_results(results_t, cumulative='avg', x_log_scale=False)

In [ ]:
plot_multiple_results(results_t, cumulative='no', x_log_scale=False, window=100)

## 4 - Experimentos com "FrozenLake-v1"

Nesta seção, vamos fazer experimentos similares aos da seção 3, mas focando no ambiente `FrozenLake`.

Este ambiente tem como diferencial o fato de ser **não-determinístico**:
- uma ação tem 1/3 de chance de dar o resultado correto
- e 1/3 de fazer o agente mover em cada direção perpendicular à desejada.

Isso afeta um pouco os resultados, como veremos.

### 4.1 - Otimizando Parâmetros do Q-Learning

In [ ]:
# Ao setar as variáveis abaixo, a função 'train' vai servir para a otimização neste ambiente
ENV = gym.make("FrozenLake-v1")
RUNS_PER_TRIAL = 7
EPISODES_PER_TRIAL = 1_000

In [ ]:
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_planning.db',
                        study_name=f'qlearning_frozen',
                        load_if_exists=True)

study.optimize(train, n_trials=50)
clear_output()

print("MELHORES PARÂMETROS:")
print(study.best_params)
qlearn_params_frozen = study.best_params

### 4.2 - Experimentos Comparativos

In [ ]:
environment = gym.make("FrozenLake-v1")
EPISODES = 7_000
RUNS = 20
AUTO_LOAD = True

results_f = []

results_f.append( repeated_exec(RUNS, f"Q-Learning ", run_qlearning, environment, EPISODES, **qlearn_params_frozen, auto_load=AUTO_LOAD) )
clear_output()

#plan_steps = 1
#results_f.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passo)", run_dyna_q, environment, EPISODES, **qlearn_params_frozen, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
#clear_output()

plan_steps = 5
results_f.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passo)", run_dyna_q, environment, EPISODES, **qlearn_params_frozen, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
clear_output()

In [ ]:
plot_multiple_results(results_f, cumulative='avg', x_log_scale=False)

In [ ]:
plot_multiple_results(results_f, cumulative='no', x_log_scale=False, window=100)

## 5 - Desafio

O Dyna-Q foi bem no ambiente `FrozenLake-v1`?

Apesar de variar entre execuções, ele não costuma ser muito bom para quantidades altas de "passos de planejamento".

**Você acha que isso aconteceu por qual motivo?**

Proponha uma modificação simples no *modelo* e no *planejamento* do Dyna-Q para melhorar o desempenho dele nesse ambiente.

Depois, refaça os experimentos da seção 4.2 para conferir se deu certo.

In [ ]:
def planning_new(model, planning_steps, Q, lr, gamma):
    # altere aqui !!!
    pass

In [ ]:
# Algoritmo Dyna Q
def run_dyna_q_new(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, planning_steps=5):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

   # inicializa a tabela Q
    Q = np.zeros(shape=(env.observation_space.n, num_actions))

    model = dict({})

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            if done:
                # para estados terminais
                V_next_state = 0
                next_state = env.reset()
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table / direct RL
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            # atualiza o modelo
            # ALTERE AQUI !!!

            # planejamento / indirect RL
            planning_new(model, planning_steps, Q, lr, gamma)

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 1000 passos, imprime informação sobre o progresso
        if VERBOSE and ((i+1) % 1000 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Step {i+1} Average Reward (last 100): {avg_reward:.3f}")

    state = env.reset()
    reward = 0

    return sum_rewards_per_ep, Q